In [1]:
# Run it once and Comment
# import sys
# sys.path.append('C:\\Shaukat\\code\\functions_implemented')
# print sys.path
# import functions_implemented as fi


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats
from datetime import datetime, timedelta

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from keras.layers.recurrent import LSTM

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from copy import deepcopy

from keras.models import load_model

%matplotlib inline





Using Theano backend.


In [3]:
df_gfs = pd.read_csv("C:\\Shaukat\\code\\data_rep\\gfs\\csv_files\\gfs_cygnet.csv")
df_bom = pd.read_csv("C:\\Shaukat\\code\\data_rep\\bom\\Cygnet.csv")

In [4]:
# Setting up GFS
df_gfs['Unnamed: 0'] = pd.to_datetime(df_gfs['Unnamed: 0'])
df_gfs.set_index('Unnamed: 0', inplace = True)

# Setting up BOM
# df_bom['date_only'] = df_bom.UtcTime.dt.date
df_bom.UtcTime = pd.to_datetime(df_bom.UtcTime)
df_bom.set_index('UtcTime', inplace=True)

# Trim df_bom and df_gfs from november 2016
df_bom = df_bom.loc['2016-11-01 00:00:00':]
df_gfs = df_gfs.loc['2016-11-01 00:00:00':]

In [5]:
df_gfs.head(4)

,pred_cloud_cover,pred_cloud_cover_bound_cloud_layer,pred_convective_cloud,pred_dewp,pred_high_tcc,pred_low_tcc,pred_lw_rad,pred_max_wind_press,pred_merid_wind,pred_middle_tcc,...,pred_surf_haines,pred_surf_momentum_vflux,pred_surface_pressure,pred_sw_rad,pred_temp,pred_total_rain,pred_ustorm,pred_vstorm,pred_wind_speed_surf,pred_zonal_wind
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2016-11-01 00:00:00,13,0,0,275.380005,0,13,270,35201.79297,-1.19,0,...,NaN,0.084,98587.53906,608,284.179993,0.14,17.930000,-9.89,13.1,8.61
2016-11-01 03:00:00,22,0,0,273.350006,0,22,277,34369.94922,-1.35,0,...,NaN,0.122,98606.57813,900,286.149994,0.03,18.320000,-10.28,13.9,8.61
2016-11-01 06:00:00,0,0,0,274.200012,0,0,261,36218.03125,-3.29,0,...,NaN,0.218,98471.00000,780,285.779999,0.00,18.700001,-12.48,13.5,6.47
2016-11-01 09:00:00,2,0,0,276.200012,0,2,263,40477.56641,-4.63,1,...,NaN,0.331,98555.23438,230,282.130005,0.02,23.870001,-12.50,16.9,7.50


In [6]:
df_bom.head(4)

,ObservationId,Timestamp,ParentLocationId,RelatedLocationId,ReadingId,LocalTime,TimeZone,Temperature,FeelsLike,DewPoint,RelativeHumidity,WindDirection,WindDirectionCompass,WindSpeed,WindGust,RainSince9am,Pressure,RainfallLastHour,RainfallLast10Minutes
UtcTime,,,,,,,,,,,,,,,,,,,
2016-11-01 00:00:00,225226,1477918800,18,94220,312884,2016-11-01 00:00:00.000,Australia/Hobart,12.1,10.3,2.7,53.0,310,NW,17,30,2.0,NaN,0.0,0.0
2016-11-01 00:10:00,225265,1477919400,18,94220,312930,2016-11-01 00:10:00.000,Australia/Hobart,11.7,9.8,2.2,52.0,316,NW,17,22,2.0,NaN,0.0,0.0
2016-11-01 00:20:00,225290,1477920000,18,94220,312976,2016-11-01 00:20:00.000,Australia/Hobart,12.0,10.2,1.7,49.0,300,WNW,17,20,2.0,NaN,0.0,0.0
2016-11-01 00:30:00,225328,1477920600,18,94220,313022,2016-11-01 00:30:00.000,Australia/Hobart,11.9,9.6,1.3,48.0,300,WNW,22,43,2.0,NaN,0.0,0.0


In [7]:
# Prepare groundtruth dataframe from BOM 
tot_gfs_entries = len(df_gfs.index)
df_gt = pd.DataFrame(index=df_gfs.index, columns=['rainfall_t+1','rainfall_t+2','rainfall_t+3'])
for dt_gfs in range(0,tot_gfs_entries):
    # Iterate each date in GFS and get the groundtruth rainfall from BOM
    
    date_a = df_gfs.index[dt_gfs] # Forecast time in gfs (made every three hours)
    date_t1 = date_a + timedelta(hours=1) # rainfall_t+1
    date_t2 = date_a + timedelta(hours=2) # rainfall_t+2
    date_t3 = date_a + timedelta(hours=3) # rainfall_t+3
    print 'processing: ', date_a 

    # df_patch for t+1 (Gather groundtruth rainfall from BOM)
    df_patch = df_bom.loc[date_a:date_t1].copy()
    rainfall_t1 = df_patch.RainfallLast10Minutes.sum()
    print 'date_t1: ', date_t1 

    
    # df_patch for t+2 (Gather groundtruth rainfall from BOM)
    df_patch = df_bom.loc[date_t1:date_t2].copy()
    rainfall_t2 = df_patch.RainfallLast10Minutes.sum()
    print 'date_t2: ', date_t2 

    
    # df_patch for t+3 (Gather groundtruth rainfall from BOM)
    df_patch = df_bom.loc[date_t2:date_t3].copy()
    rainfall_t3 = df_patch.RainfallLast10Minutes.sum()
    print 'date_t3: ', date_t3, '\n' 

    
    # Add it to the groundtruth df
    df_gt.loc[date_a] = [rainfall_t1, rainfall_t2, rainfall_t3]
    
    # Delete variables
    del date_a
    del date_t1
    del date_t2
    del date_t3
    del rainfall_t1
    del rainfall_t2
    del rainfall_t3
    

processing:  2016-11-01 00:00:00
date_t1:  2016-11-01 01:00:00
date_t2:  2016-11-01 02:00:00
date_t3:  2016-11-01 03:00:00 

processing:  2016-11-01 03:00:00
date_t1:  2016-11-01 04:00:00
date_t2:  2016-11-01 05:00:00
date_t3:  2016-11-01 06:00:00 

processing:  2016-11-01 06:00:00
date_t1:  2016-11-01 07:00:00
date_t2:  2016-11-01 08:00:00
date_t3:  2016-11-01 09:00:00 

processing:  2016-11-01 09:00:00
date_t1:  2016-11-01 10:00:00
date_t2:  2016-11-01 11:00:00
date_t3:  2016-11-01 12:00:00 

processing:  2016-11-01 12:00:00
date_t1:  2016-11-01 13:00:00
date_t2:  2016-11-01 14:00:00
date_t3:  2016-11-01 15:00:00 

processing:  2016-11-01 15:00:00
date_t1:  2016-11-01 16:00:00
date_t2:  2016-11-01 17:00:00
date_t3:  2016-11-01 18:00:00 

processing:  2016-11-01 18:00:00
date_t1:  2016-11-01 19:00:00
date_t2:  2016-11-01 20:00:00
date_t3:  2016-11-01 21:00:00 

processing:  2016-11-01 21:00:00
date_t1:  2016-11-01 22:00:00
date_t2:  2016-11-01 23:00:00
date_t3:  2016-11-02 00:00:00 



In [9]:
# df_gt.to_csv("check_cygnet_gt_rainfall.csv")
# X: df_gfs
# Y: df_gt

In [11]:
# df_gt

In [12]:
# print df_gfs.columns

In [13]:
col_list = ['pred_cloud_cover', 'pred_cloud_cover_bound_cloud_layer',
       'pred_convective_cloud', 'pred_dewp', 'pred_high_tcc',
       'pred_low_tcc', 'pred_lw_rad', 'pred_max_wind_press',
       'pred_merid_wind', 'pred_middle_tcc', 'pred_rain_rate',
       'pred_rel_humidity', 'pred_sunshine', 'pred_surf_geowind', 'pred_surf_momentum_vflux', 'pred_surface_pressure', 'pred_sw_rad',
       'pred_temp', 'pred_total_rain', 'pred_ustorm', 'pred_vstorm',
       'pred_wind_speed_surf', 'pred_zonal_wind']
df_X = df_gfs[col_list].copy()
df_Y = df_gt.copy()

In [14]:
# Make sure that df_X is not null
df_X_bool = df_X.isnull().any().any()
print df_X_bool, 'THIS MUST BE FALSE' # This must be FALSE

# df_X.isnull().any()

False THIS MUST BE FALSE


In [15]:
# df_Y

In [16]:
# def prepare_lstm_training_test_set(data_matrix, label_matrix):
#     sliding_window = 3
#     tot_samples = data_matrix.shape[0]
#     sample_dimension = data_matrix.shape[1]
#     print "Total Sample: ",tot_samples
#     print "Dimension of each sample: ",sample_dimension
    
#     # Accumulate examples here in this list
#     input_lstm = []
    
#     for sequence in range(0,tot_samples-sliding_window):
#         input_lstm.append(data_matrix[sequence:sequence+sliding_window, :])
        
    
    
    

In [17]:
df_data_matrix = df_X.as_matrix()
df_gt_rainfall_forecast_matrix = df_Y.as_matrix()
print 'Shape of data matrix: ',df_data_matrix.shape
print 'Shape of label matrix: ',df_gt_rainfall_forecast_matrix.shape

# Normalize data matrix with zero mean and unit covariance
# http://scikit-learn.org/stable/modules/preprocessing.html
df_data_matrix = preprocessing.scale(df_data_matrix)

data_matrix = np.copy(df_data_matrix)
label_matrix = np.copy(df_gt_rainfall_forecast_matrix)

sliding_window = 3
tot_samples = data_matrix.shape[0]
sample_dimension = data_matrix.shape[1]
print "Total Sample: ",tot_samples
print "Dimension of each sample: ",sample_dimension
    
# Accumulate examples here in this list
input_lstm = []
label_lstm = []
label_entry = sliding_window

    
for sequence in range(0,tot_samples-sliding_window):
    input_lstm.append(data_matrix[sequence:sequence+sliding_window, :])
    label_lstm.append(label_matrix[label_entry,0:3])
    print "sequence, sequence+sliding_window, label_entry: ",sequence,sequence+sliding_window,label_entry
    label_entry = label_entry + 1


return_data_matrix_lstm = np.array(input_lstm)
return_label_matrix_lstm = np.array(label_lstm)

print "LSTM input: ",return_data_matrix_lstm.shape
print "LSTM output: ",return_label_matrix_lstm.shape

Shape of data matrix:  (809L, 23L)
Shape of label matrix:  (809L, 3L)
Total Sample:  809
Dimension of each sample:  23
sequence, sequence+sliding_window, label_entry:  0 3 3
sequence, sequence+sliding_window, label_entry:  1 4 4
sequence, sequence+sliding_window, label_entry:  2 5 5
sequence, sequence+sliding_window, label_entry:  3 6 6
sequence, sequence+sliding_window, label_entry:  4 7 7
sequence, sequence+sliding_window, label_entry:  5 8 8
sequence, sequence+sliding_window, label_entry:  6 9 9
sequence, sequence+sliding_window, label_entry:  7 10 10
sequence, sequence+sliding_window, label_entry:  8 11 11
sequence, sequence+sliding_window, label_entry:  9 12 12
sequence, sequence+sliding_window, label_entry:  10 13 13
sequence, sequence+sliding_window, label_entry:  11 14 14
sequence, sequence+sliding_window, label_entry:  12 15 15
sequence, sequence+sliding_window, label_entry:  13 16 16
sequence, sequence+sliding_window, label_entry:  14 17 17
sequence, sequence+sliding_window, 

In [18]:
# Prepare test and train points
tot_points = return_data_matrix_lstm.shape[0]
train_points = int(np.floor(0.6*tot_points)) 
test_points = tot_points - train_points
print 'tot_points: ', tot_points, ' train_points: ', train_points, ' test_points: ', test_points

# Generate Train Sequence
x_train = np.copy(return_data_matrix_lstm[0:train_points + 1,:])
y_train = np.copy(return_label_matrix_lstm[0:train_points + 1,:])
# Generate Test Sequence
x_test = np.copy(return_data_matrix_lstm[train_points + 1:tot_points + 1 ,:])
y_test = np.copy(return_label_matrix_lstm[train_points + 1:tot_points + 1 ,:])

print 'x_train_shape: ', x_train.shape, ' y_train_shape: ', y_train.shape
print 'x_test_shape: ', x_test.shape, ' y_test_shape: ', y_test.shape

tot_points:  806  train_points:  483  test_points:  323
x_train_shape:  (484L, 3L, 23L)  y_train_shape:  (484L, 3L)
x_test_shape:  (322L, 3L, 23L)  y_test_shape:  (322L, 3L)


In [21]:
y_train[0:3]

array([[0.20000000000000001, 0.20000000000000001, 0.20000000000000001],
       [0.40000000000000002, 0.20000000000000001, 3.6000000000000005],
       [0.20000000000000001, 2.3999999999999999, 0.20000000000000001]], dtype=object)

In [ ]:
x_train.shape

In [ ]:
x_train.shape[2]

In [ ]:
y_train.shape

In [ ]:
y_train.shape[1]

In [ ]:
input_dim = x_train.shape[2] # Input vector dimension for LSTM (N x timesteps x dim of seq)
output_dim = y_train.shape[1] # Output dimension for LSTM (temperature for t+1,t+2,t+3)

# create model
model = Sequential()
layers = [input_dim, 50, 100, output_dim]

model.add(LSTM(input_dim=layers[0],output_dim=layers[1],return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(layers[2], return_sequences=False))
model.add(Dropout(0.1))

model.add(Dense(output_dim=layers[3]))
model.add(Activation("linear"))
    
# compile model
# start = time.time()
model.compile(loss="mse", optimizer="rmsprop")
# print "Compilation Time : ", time.time() - start
# model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# filepath_savemodel = 'C:\\Shaukat\\code\\rainfall\\model_stored\\weights.{epoch:02d}-{val_loss:.2f}.h5'
filepath_savemodel = 'C:\\Shaukat\\code\\rainfall\\model_stored\\model_lstm_v2.h5'
# filepath_csvlogger = 'C:\\Shaukat\\code\\rainfall\\model_stored\\training_s2.log'

checkpointer = ModelCheckpoint(filepath_savemodel, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
# csv_logger = CSVLogger(filepath_csvlogger, separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0.001)

# history = model.fit(x_train, y_train, nb_epoch=1000, verbose=1, validation_data=(x_test, y_test), batch_size = 32, shuffle=True, callbacks=[checkpointer, csv_logger, reduce_lr])
history = model.fit(x_train, y_train, nb_epoch=500, verbose=1, validation_data=(x_test, y_test), batch_size = train_points, shuffle=True, callbacks=[checkpointer, reduce_lr])



In [ ]:
model.summary()

In [ ]:
vec_test = y_test.flatten()
vec_test = vec_test.reshape((len(vec_test),1))
print vec_test.shape
print 'total test points: ', vec_test.shape[0]

predictions = model.predict(x_test)
pred_vec = predictions.flatten()
pred_vec = pred_vec.reshape((len(pred_vec),1))
print pred_vec.shape

In [ ]:
# Preprocess block
# threshold = 1.0
# preproc_y = deepcopy(pred_vec)
# preproc_y[preproc_y<=threshold] = 0.0

In [ ]:
# For plot
time_index = np.arange(0,vec_test.shape[0],1)
time_index = time_index.reshape(((len(time_index),1)))

plt_lower_limit = 0
plt_upper_limit = 100

plt.figure(figsize=(25,8))

# plot

# Full plot
# plt.plot(time_index,vec_test,label="GroundTruth",color='b')
# plt.plot(time_index,pred_vec,label="Prediction",color='g')

# Unpreprocessed Plot
plt.plot(time_index[plt_lower_limit:plt_upper_limit],vec_test[plt_lower_limit:plt_upper_limit],label="GroundTruth",color='b')
plt.plot(time_index[plt_lower_limit:plt_upper_limit],pred_vec[plt_lower_limit:plt_upper_limit],label="Prediction",color='g')

# Preprocessed Plot
# plt.plot(time_index[plt_lower_limit:plt_upper_limit],vec_test[plt_lower_limit:plt_upper_limit],label="GroundTruth",color='b')
# plt.plot(time_index[plt_lower_limit:plt_upper_limit],preproc_y[plt_lower_limit:plt_upper_limit],label="Prediction",color='g')

plt.xlabel('Time')
plt.ylabel('Rainfall prediction in mm')
plt.legend()
plt.title('Groundtruth and Predicted Rainfall per hour')
# img_filename = 'C:\\Users\\ShaukatAbidi\\Documents\\shaukat_python_progs\\learning\\time_series\\feed_forward_nn\\models\\s7_images\\'+str(test_ex)+'.png'
# plt.savefig(img_filename, bbox_inches='tight')
# plt.clf()